In [ ]:
%pip install numpy nltk 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [518]:
from pkgs import txt

import numpy as np

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [519]:
file1 = open('doc1.txt','r')
file2 = open('doc2.txt','r')

a = file1.read()
b = file2.read()

In [520]:
lemmatizer = WordNetLemmatizer()
stop = set(stopwords.words("english"))
stop.remove("not")
stop.remove("no")
posIgnore = ["$","''","(",")",",","--",".",":","CD","FW","LS","NNP","NNPS","NNS","PRP","PRP$","SYM","TO","WDT","WP","WP$","WRB","``"]
exStr = "There is a setuid binary in the homedirectory that does the following: it makes a connection to localhost on the port you specify as a commandline argument. It then reads a line of text from the connection and compares it to the password in the previous level."

In [521]:
def getWnetPos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def simplify(parag = exStr):
    stns = sent_tokenize(parag)
    wrds = []
    for stn in stns:
        wrds.append(word_tokenize(stn))
    wrdsFilt = []
    for i in wrds:
        temp = []
        for j in i:
            if j.casefold() not in stop:
                temp.append(j)
        wrdsFilt.append(temp)

    tagged = []
    for words in wrdsFilt:
        tagged.append(nltk.pos_tag(words))
    wrdsLem = []
    for words in tagged:
        temp = []
        for word in words:
            if word[1] in posIgnore:
                temp.append((word[0],word[1]))    
            elif getWnetPos(word[1]) != None:
                temp.append((lemmatizer.lemmatize(word[0],pos=getWnetPos(word[1])),word[1]))
            else:
                temp.append((lemmatizer.lemmatize(word[0]),word[1]))
        wrdsLem.append(temp)
    
    syn = []
    for words in wrdsLem:
        temp = []
        for word,tag in words:
            if word in ['.',',']:
                continue
            temp.append((word,tag))
        syn.append(temp)

    return syn

In [522]:
def nMax(listIn, n):
    maxN = []
    list1 = listIn.copy()
    for i in range(0, min(n,len(listIn))): 
        max1 = list1[0]
        for e in list1:     
            if e > max1:
                max1 = e  
        
        list1.remove(max1)
        maxN.append(max1)
    return maxN
    
def nMin(listIn, n):
    minN = []
    list1 = listIn.copy()
    for i in range(0, min(n,len(listIn))): 
        min1 = list1[0]
        for e in list1:     
            if e < min1:
                min1 = e  
        
        list1.remove(min1)
        minN.append(min1)
    return minN

In [523]:
from itertools import product
import nltk
from nltk.corpus import wordnet

def wordSimilarity(wrd1,wrd2):
    
    syns1 = wordnet.synsets(wrd1)
    syns2 = wordnet.synsets(wrd2)
    for sn1, sn2 in product(syns1, syns2):
        simil = wordnet.wup_similarity(sn1, sn2)
        if syns1 == syns2:
            return 1.0
        else:
            return simil #if (simil != None) else 0.5 # Due to None implying uncertainity


def comp(txt1,txt2):
    #Word Comparasion and Scoring
    totalScore=[]
    mismatch=[]
    for sA,sB in product(txt1,txt2):
        discard = []
        score = []
        for wA in sA:
            wrdScores = []
            if wA in discard: continue
            for wB in sB:
                if wB in discard: continue
                wrdScores.append(wordSimilarity(wA[0],wB[0])) 
                #print(wA[0],wB[0],wordSimilarity(wA[0],wB[0]))
            #print(wrdScores)
            wrdScores = [i if i != None else 0.1 for i in wrdScores] # Replace all uncertain values, ie. None, with a valid approximate value
            score.append(max(wrdScores))
            discard.append(sB[wrdScores.index(max(wrdScores))])
        totalScore.append((sum(nMax(score,min(len(sA),len(sB))))/len(nMax(score,min(len(sA),len(sB)))),(txt1.index(sA),txt2.index(sB))))
        mismatch.append((sum(nMin(score,min(3,len(score))))/len(nMin(score,min(3,len(score)))),(txt1.index(sA),txt2.index(sB))))
    
    scores = []
    temp = []
    if(len(txt1)>len(txt2)):
        for i in range(0,len(txt2)):
            temp.append([])
        for el in totalScore:
            temp[el[1][1]].append(el) 
    else:
        for i in range(0,len(txt1)):
            temp.append([])
        for el in totalScore:
            temp[el[1][0]].append(el) 
    for a in temp:
        scores.append(max(a))
            
    return scores,nMin(mismatch,3)

In [524]:
def result(scores):
    res = []
    for el in scores:
        res.append((1 if el[0]>0.7 else (0.5 if el[0]>0.35 else 0),el[1]))
    return res

In [525]:
scores,mis = comp(simplify(a),simplify(b))
res = result(scores)

score100 = sum([1 if a[0] == 1 else 0 for a in res])/len([1 if a[0] == 1 else 0 for a in res])
score50 = sum([1 if a[0] >= 0.5 else 0 for a in res])/len([1 if a[0] >= 0.5 else 0 for a in res])
score0 = sum([1 if a[0] < 0.5 else 0 for a in res])/len([1 if a[0] < 0.5 else 0 for a in res])

print("100% Similar: ",score100*100,"%")
print("Atleast 50% Similar: ",score50*100,"%")
print("100% Dissimilar: ",score0*100,"%")
print("Top 3 Mismatches:")
print()

for m in mis:
    print("In First Test:")
    print(sent_tokenize(a)[m[1][0]])
    print("In Second Test:")
    print(sent_tokenize(a)[m[1][1]])
    print()



100% Similar:  40.0 %
Atleast 50% Similar:  100.0 %
100% Dissimilar:  0.0 %
Top 3 Mismatches:

In First Test:
This characteristic greatly contrasted with contemporary digital encyclopedias such as Microsoft Encarta, Encyclopedia Britannica, and even Bomis's Nupedia, which was Wikipedia's direct predecessor.
In Second Test:
Crucially, Stallman's concept specifically included the idea that no central organization should control editing.

In First Test:
This characteristic greatly contrasted with contemporary digital encyclopedias such as Microsoft Encarta, Encyclopedia Britannica, and even Bomis's Nupedia, which was Wikipedia's direct predecessor.
In Second Test:
This characteristic greatly contrasted with contemporary digital encyclopedias such as Microsoft Encarta, Encyclopedia Britannica, and even Bomis's Nupedia, which was Wikipedia's direct predecessor.

In First Test:
This characteristic greatly contrasted with contemporary digital encyclopedias such as Microsoft Encarta, Encyclope